## KNN classifier in Iris and robot dataset

# Use of KNN classifier
     1.Getting all neighbours for that test point
         neighbours=get_neighbors(train_dataset, test_sample,k)
	 2.Extracting class with highest number of votes among those neighbours
         major_class= get_votes(neighbors)
     3.appending classfication for that result
         prediction_list.append(major_class)

In [340]:
import pandas as pd
import numpy as np
import matplotlib as mlt
import csv
import random
import operator
import math
from sklearn.metrics.pairwise import cosine_similarity as cos
from sklearn.metrics import precision_recall_fscore_support as score

In [341]:
train_df=[]
validation_df=[]
split = 0.2
df=pd.read_csv('Iris.csv',header=None)
dataset=list(df)
df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,6.3,3.3,6.0,2.5,Iris-virginica
2,5.0,3.0,1.6,0.2,Iris-setosa
3,6.2,2.8,4.8,1.8,Iris-virginica
4,6.4,3.1,5.5,1.8,Iris-virginica


### splitting data as train and validation

In [342]:
def divide(df,validation_data_size):
    indices = df.index.tolist()
    validation_data_size=round(validation_data_size*len(df))
    validation_indices = random.sample(population=indices, k=validation_data_size)
    train_data = df.drop(validation_indices)
    validation_data = df.loc[validation_indices]
    return train_data,validation_data

In [343]:
random.seed(0)
train_df,validation_df = divide(df,validation_data_size=0.2)

### Converting dataframe into list

In [344]:
train_data=train_df.values
validation_data=validation_df.values
print(type(validation_data))
print(validation_data[:,-1])

<class 'numpy.ndarray'>
['Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-virginica']


### getting Euclidean  distance,manhattan distance and cosine similarity between test entry and train entry(both input as list)

In [350]:
def get_euclidean_distance(test_entry,train_entry):
    distance = 0
    # we should neglect last entry which is target label
    for i in range(len(train_entry)-1):
        distance += pow((test_entry[i]-train_entry[i]),2)
    return math.sqrt(distance)

def get_cosine_similarity(test_entry,train_entry):
    train=np.array(train_entry[:-1])
    test=np.array(test_entry[:-1])
    dot_product = np.dot(test,train)
    norm_train = np.linalg.norm(train)
    norm_test = np.linalg.norm(test)
    return 1-(dot_product / (norm_train * norm_test))

def get_manhattan_distance(test_entry,train_entry):
    distance=0
    for i in range(len(train_entry)-1):
        distance += abs(test_entry[i]-train_entry[i])
    return math.sqrt(distance)

### Getting neighbours for the test instance (one row from test)

In [352]:
def get_neighbours(train_data,test_entry,k):
    neighbour_distances = []
    for train_entry in train_data:
        #dist = get_euclidean_distance(test_entry,train_entry)
        #dist=get_manhattan_distance(test_entry,train_entry)
        dist=get_cosine_similarity(test_entry,train_entry)
        neighbour_distances.append((train_entry,dist))
    neighbour_distances.sort(key=operator.itemgetter(1))
    k_closest_neighbors = []
    for i in range(k):
        #saving the neighbors entries so we can extract the last column while deciding major class in 
        #get_votes function
        k_closest_neighbors.append(neighbour_distances[i][0])
    return k_closest_neighbors
    

###  get major feature from k_closest_neighbours

In [353]:
def get_votes(neighbors):
    votes_dict = {}
    for i in neighbors:
        v=i[-1]
        if v in votes_dict:
            votes_dict[v] += 1
        else:
            votes_dict[v]=1
    sorted_votes = sorted(votes_dict.items(), key=operator.itemgetter(1),reverse=True)
    return sorted_votes[0][0]

### get_accuracy function takes input as classified list and actual list

In [354]:
def get_accuracy(pred,actual):
    a,b,c,d=score(actual,pred)  
    
    #calculation of precision ,recall and F1 score
    print("precision:",a)
    print("recall:",b)
    print("fscore:",c)
    return a

    

### Test the  above functions

In [370]:
#User value of k=3
k=3

prediction_list=[]
count=0
entries=0
for test in validation_data:
    neighbours=get_neighbours(train_data,test,8)
    major_class = get_votes(neighbours)
    prediction_list.append(major_class)
    print('classified:',major_class)
    print("actual label:",test[-1])
    if(major_class==test[-1]):
        print("Correct classification")
        count+=1
    else:
        print("Incorrect classification")
    entries+=1
print("Tests:",entries)
print("Correct:",count)
print("Accuracy:",(count/entries))
#print(type(np.asarray(prediction_list)))
#print(type(validation_data[:,-1]))
accuracy = get_accuracy(np.array(prediction_list),validation_data[:,-1])
print("Accuracy:",accuracy);

classified: Iris-versicolor
actual label: Iris-versicolor
Correct classification
classified: Iris-virginica
actual label: Iris-virginica
Correct classification
classified: Iris-setosa
actual label: Iris-setosa
Correct classification
classified: Iris-versicolor
actual label: Iris-versicolor
Correct classification
classified: Iris-virginica
actual label: Iris-virginica
Correct classification
classified: Iris-setosa
actual label: Iris-setosa
Correct classification
classified: Iris-versicolor
actual label: Iris-versicolor
Correct classification
classified: Iris-versicolor
actual label: Iris-versicolor
Correct classification
classified: Iris-setosa
actual label: Iris-setosa
Correct classification
classified: Iris-setosa
actual label: Iris-setosa
Correct classification
classified: Iris-setosa
actual label: Iris-setosa
Correct classification
classified: Iris-versicolor
actual label: Iris-versicolor
Correct classification
classified: Iris-setosa
actual label: Iris-setosa
Correct classification

## KNN Model on admission dataset

In [375]:
admission_train_df=[]
admission_validation_df=[]
split= 0.2
df=pd.read_csv('admission_data.csv')
dataset=list(df)
df.drop(df.columns[1], axis=1)
df.head()

KeyError: '[242 334 4 45 232 167 166 196 147 389 282 426 478 374 214 90 142 258 406\n 457 317 294 162 8 500 274 11 455 35 143 422 159 163 357 289 375 241 78\n 486 102 27 254 81 484 435 93 204 442 182 2 169 227 430 262 307 293 52 37\n 371 299 253 404 310 496 248 135 50 447 420 150 383 73 270 117 247 108 459\n 252 74 205 286 431 57 257 25 395 353 63 482 139 255 6 126 211 285 144 287\n 238 461 28 360 386 140 79 92 151 58 3 44 234 136 174 456 17 354 275 265\n 421 56 398 172 410 207 445 263 141 34 113 225 22 42 379 206 414 46 190\n 228 62 53 438 230 352 185 15 358 266 244 330 173 463 370 272 367 305 134\n 466 490 271 109 101 306 425 318 192 105 165 460 224 157 218 290 387 325\n 32 302 408 409 324 429 104 40 246 193 183 277 394 124 154 418 68 451 107\n 405 342 311 55 250 19 450 75 493 122 304 308 229 100 39 384 441 48 341 54\n 233 292 499 41 333 340 29 33 12 469 390 249 497 261 453 436 223 76 106\n 320 188 476 396 116 10 348 235 61 361 88 498 403 201 319 231 181 393 222\n 111 213 480 416 210